In [2]:
# setup
from pyswmm import Simulation
import os
import dask
import time

In [3]:
# nsims
nsims = 2

In [4]:
# specify location
print(os.path.abspath(os.curdir))
os.chdir("..")
os.chdir("probabilistic_python")
dir_path = os.path.abspath(os.curdir)
print(dir_path)

input_path = dir_path + r'\input\swmm\input_'
print(input_path)

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\jupyter_notebooks
C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python
C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_


In [5]:
@dask.delayed
def delay_job(i, lib_path=None):
    sim_folder = input_path + str(i)
    sim_file = os.path.join(sim_folder, r'JS_NPlesantCreek.inp')
    print(sim_file)

    binary_path = sim_folder + r'\JS_NPlesantCreek.out'

    # delete pre-existing .out in order to run swmm
    if os.path.exists(binary_path):
        os.remove(binary_path)
    else:
        print("Can not delete the file as it doesn't exist")

    # load the model - no interaction, write out binary file
    sim = Simulation(inputfile=sim_file, reportfile=None, outputfile=binary_path, swmm_lib_path=lib_path)
    #return(sim)
    sim.execute()
    return("file " + str(i) + " simulated!")

In [6]:
# can it do 1?
delayed_job = [delay_job(1, lib_path = 'C:/ProgramData/Anaconda3/envs/epa/lib/site-packages/pyswmm/lib/windows/swmm5-x64.dll')]
delayed_job

[Delayed('delay_job-a56007ca-8247-49a0-9b9d-fe0e76f77488')]

In [7]:
%%time
dask.delayed(print)(delayed_job).compute()

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp

... SWMM Version 5.3.0.dev0
... Run Complete['file 1 simulated!']
Wall time: 55.3 s


In [8]:
# can it do multiple?
#delayed_jobs = [delay_job(a) for a in range(1,nsims+1)]
delayed_jobs = [delay_job(1, lib_path = 'C:/ProgramData/Anaconda3/envs/epa/lib/site-packages/pyswmm/lib/windows/swmm5-x64.dll'),
                delay_job(2, lib_path = 'C:/ProgramData/Anaconda3/envs/epa/lib/site-packages/pyswmm/lib/windows/swmm5-x64-copy.dll')]
delayed_jobs

[Delayed('delay_job-98939c36-f21d-4fa5-9f47-acb8719052a6'),
 Delayed('delay_job-15c655e1-d788-416d-b953-7ee6260fe512')]

In [9]:
%%time
dask.delayed(print)(delayed_jobs).compute()

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_2\JS_NPlesantCreek.inp
C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp
Can not delete the file as it doesn't exist

... SWMM Version 5.3.0.dev0
... SWMM Version 5.3.0.dev0
... Run Complete
... Run Complete['file 1 simulated!', 'file 2 simulated!']
Wall time: 2min 23s


Unfortunately, it looks like, even though python and dask can handle simultaneously running two parallel processes, pyswmm cannot. I don't know if that is a problem that can be fixed with multiple computers, a different package, or at all. 